# Data wrangling with MongoDB

## 3. Visão geral dos dados

### 3.1	Representação dos dados OSM 

Os dados estão organizados em uma estrutura de elementos do mapa que representam o objeto e seu valor associado. Tipos encontrados:

- node: um nó é informações de um par de coordenadas, sendo latitude e longitude;
- way : uma lista de nós (nodes);

Apesar do OpenStreetMap ter mais dois tipos de elementos abaixo listado, eles não foram considerados neste projeto.
- area: representa uma área limitada, por exemplo, uma praça;
- relation: pode ser um conjunto de nodes, ways e relation relacionados.

In [42]:
import pprint

### 3.2 Importação dos dados para pesquisa 

O arquivo mapbh.json, gerado após análise dos dados OSM para Belo Horizonte, foi importado para o MongoDB através da linha de comando:

* => mongoimport -h 127.0.0.1:27017 --db openstreetmap --collection mapbh --file C:/Nanodegree/MongoDb/Trab_final/mapbh.json  


Os dados no MongoDB serão acessados e manipulados através da biblioteca pymongo.

In [ ]:
import pymongo
from pymongo import MongoClient
client=MongoClient('localhost:27017')
db=client.openstreetmap

Os dados importados fazem parte de uma coleção nomeada mapbh (--collection mapbh). 
Verificando se o nome foi criado corretamente: 

In [29]:
db.collection_names()

[u'mapbh']

### 3.3 Conhecendo os dados 

** 1. Total de documentos **

In [30]:
db.mapbh.find().count()

466990

** 2. Número de documentos com o tipo "node": **

In [38]:
db.mapbh.find({"type":"node"}).count() 

408052

** 3. Número de documentos com o tipo "way": **

In [39]:
db.mapbh.find({"type":"way"}).count()

58875

** 4. Documentos com tipo diferente de "way" e "node": **

O total de node e way é (408.052 + 58.875) 466.927, um pouco diferente dos 466.990 listados no db.mapbh.find().count(). Vou listar esses registros que não são way, nem node para identificar o problema, visto que na função shape_element eu fitrei os registros por tags que contivessem as strings "node" e "way".

In [44]:
pprint.pprint(list(db.mapbh.find({"type":{"$nin":["way","node"]}})))

[{u'_id': ObjectId('58641664ed425d6e306c37b3'),
  u'amenity': u'Public_Building',
  u'created': {u'changeset': u'10870093',
               u'timestamp': u'2012-03-04T15:52:56Z',
               u'uid': u'397850',
               u'user': u'Tulio Magno Quites Machado Filho',
               u'version': u'3'},
  u'historic': u'Monument',
  u'id': u'317211433',
  u'name': u'Coreto',
  u'natural': u'Tree',
  u'pos': [-19.932131, -43.9379339],
  u'type': u'Palm'},
 {u'_id': ObjectId('58641666ed425d6e306d112d'),
  u'created': {u'changeset': u'10870093',
               u'timestamp': u'2012-03-04T15:52:49Z',
               u'uid': u'397850',
               u'user': u'Tulio Magno Quites Machado Filho',
               u'version': u'1'},
  u'id': u'1659158895',
  u'natural': u'Tree',
  u'pos': [-19.9312116, -43.9378039],
  u'type': u'Palm'},
 {u'_id': ObjectId('58641666ed425d6e306d112e'),
  u'created': {u'changeset': u'10870093',
               u'timestamp': u'2012-03-04T15:52:49Z',
               u

Percorrendo os registros acima é possível perceber que eles tem uma tag "type", como não foi implementada nenhum método que filtrasse esse tipo de situação, o valor desta tag própria substituiu o da tag "type" que deveria ter o tipo do atributo tag.

Estes registros indevidos serão excluídos da coleção:

In [47]:
db.mapbh.delete_many({"type":{"$nin":["way","node"]}})

Será que agora todos os registros são to tipo node ou way? Vamos conferir se o total de documentos na coleção é igual ao total de registros com way + node.

In [50]:
db.mapbh.find().count() == db.mapbh.find({"type":{"$in":["way","node"]}}).count()

True

** 5. Tamanho dos arquivos OSM e JSON **

In [ ]:
pipeline=[
            {"$group": {"_id": None, "cities":{"$addToSet":"$address.city"}}},
            {"$unwind": "$cities"},
            {"$group": { "_id": "Total City Names in the data", "count": { "$sum": 1 } } }
          ]
a =  db.mapbh.aggregate(pipeline)

In [ ]:
from bson.son import SON
pipeline = [
            {"$group":{"_id":"$tags", "total": {"$sum": {"$abs": "$value"}}}},
            {"$sort": SON([("total", -1), ("_id", -1)])}
            ]  

N=10
groups=[]
values=[]
for item in list(db.mapbh.aggregate(pipeline))[:N]:
    print(item)
    groups.append(item['_id'][0]+"_"+item['_id'][1])
    values.append(item['total'])